* **Бабижаев Илья ИУ6-54Б** Вариант 2

In [23]:
from pyspark.sql import SparkSession
import os
import math
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import expr, mean, stddev, max

In [24]:
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home'
os.environ['PATH'] = f'{os.environ['JAVA_HOME']}/bin:{os.environ['PATH']}'
os.environ['_JAVA_OPTIONS'] = '-Dhadoop.security.authentication=simple -Duser.name=spark'


spark = SparkSession.builder.appName('hw2').getOrCreate()

In [25]:
df = spark.read \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .option('recursiveFileLookup', 'true') \
    .csv('/Users/ilya/Downloads/2014-citibike-tripdata')

print(df.columns)

['tripduration', 'starttime', 'stoptime', 'start station id', 'start station name', 'start station latitude', 'start station longitude', 'end station id', 'end station name', 'end station latitude', 'end station longitude', 'bikeid', 'usertype', 'birth year', 'gender']


In [26]:
df.select('*').show()

+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|          starttime|           stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|
+------------+-------------------+-------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|         404|2014-07-01 00:00:04|2014-07-01 00:06:48|             545|     E 23 St & 1 Ave|             40.736502|           -73.97809472|           402|  Broadway & E 22 St|          40.7403432|         -73.98955109| 19578|Subscriber

In [27]:
@udf(DoubleType())
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * \
        math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

In [28]:
df_distance = df.select(
    col('start station latitude'),
    col('start station longitude'),
    col('end station latitude'),
    col('end station longitude')
).where(
    col('start station name') != col('end station name')
).withColumn(
    'distance_km',
    haversine_distance(
        col('start station latitude'),
        col('start station longitude'),
        col('end station latitude'),
        col('end station longitude')
    )
)

df_distance.show(5)

+----------------------+-----------------------+--------------------+---------------------+------------------+
|start station latitude|start station longitude|end station latitude|end station longitude|       distance_km|
+----------------------+-----------------------+--------------------+---------------------+------------------+
|             40.736502|           -73.97809472|          40.7403432|         -73.98955109|1.0555036179098545|
|            40.7361967|           -74.00859207|           40.751396|           -74.005226|1.7137101463147175|
|           40.73781509|           -73.99994661|         40.71534825|         -73.96024116| 4.175617385504693|
|           40.71146364|           -74.00552427|         40.70569254|         -74.01677685|1.1451687588860764|
|           40.73652889|           -74.00618026|         40.75044999|         -73.99481051|1.8203424930032215|
+----------------------+-----------------------+--------------------+---------------------+------------------+
o

In [29]:
df_distance.agg(
    max('distance_km').alias('max_distance'),
    mean('distance_km').alias('mean_distance'),
    stddev('distance_km').alias('stddev_distance'),
    expr('percentile_approx(distance_km, 0.5)').alias('median_distance')
).show()

+------------------+-----------------+------------------+-----------------+
|      max_distance|    mean_distance|   stddev_distance|  median_distance|
+------------------+-----------------+------------------+-----------------+
|10.629987886486502|1.704506605719486|1.1913299419591568|1.371912743772673|
+------------------+-----------------+------------------+-----------------+

